In [1]:
import json
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sqldf

## Step 1: Getting the Article and Population Data

In [2]:
page_data = pd.read_csv('page_data.csv')
WPDS_2020_data = pd.read_csv('WPDS_2020_data.csv')

page_data


,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


## Step 2: Cleaning the Data

In [3]:
new_page = page_data[page_data['page'].str[0:9] != 'Template:']
new_page.reset_index(drop = True,inplace = True)
new_page

,page,country,rev_id
0,Bir I of Kanem,Chad,355319463
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
2,Yos Por,Cambodia,393822005
3,Julius Gregr,Czech Republic,395521877
4,Edvard Gregr,Czech Republic,395526568
...,...,...,...
46696,Yahya Jammeh,Gambia,807482007
46697,Lucius Fairchild,United States,807483006
46698,Fahd of Saudi Arabia,Saudi Arabia,807483153
46699,Francis Fessenden,United States,807483270


In [4]:
WPDS_2020_data

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000
...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000
230,SB,Solomon Islands,Country,2019,0.715,715000
231,TO,Tonga,Country,2019,0.099,99000
232,TV,Tuvalu,Country,2019,0.010,10000


### Manipulating data for later 

In [8]:
keepLater = WPDS_2020_data[WPDS_2020_data['Name'].str.isupper()]
keepLater.reset_index(inplace = True)
keepLater[2:][:]

,index,FIPS,Name,Type,TimeFrame,Data (M),Population
2,2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,10,WESTERN AFRICA,WESTERN AFRICA,Sub-Region,2019,401.115,401115000
4,27,EASTERN AFRICA,EASTERN AFRICA,Sub-Region,2019,444.970,444970000
5,48,MIDDLE AFRICA,MIDDLE AFRICA,Sub-Region,2019,179.757,179757000
6,58,SOUTHERN AFRICA,SOUTHERN AFRICA,Sub-Region,2019,67.732,67732000
7,64,NORTHERN AMERICA,NORTHERN AMERICA,Sub-Region,2019,368.193,368193000
8,67,LATIN AMERICA AND THE CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Sub-Region,2019,651.036,651036000
9,68,CENTRAL AMERICA,CENTRAL AMERICA,Sub-Region,2019,178.611,178611000
10,77,CARIBBEAN,CARIBBEAN,Sub-Region,2019,43.233,43233000
11,95,SOUTH AMERICA,SOUTH AMERICA,Sub-Region,2019,429.191,429191000


In [10]:
#WPDS_2020_data.drop(WPDS_2020_data[WPDS_2020_data['Name'].str[1].str.isupper()].index, inplace=True)
WPDS_2020_data.reset_index(inplace = True)
WPDS_2020_data

,index,FIPS,Name,Type,TimeFrame,Data (M),Population
0,0,WORLD,WORLD,World,2019,7772.850,7772850000
1,1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,3,DZ,Algeria,Country,2019,44.357,44357000
4,4,EG,Egypt,Country,2019,100.803,100803000
...,...,...,...,...,...,...,...
229,229,WS,Samoa,Country,2019,0.200,200000
230,230,SB,Solomon Islands,Country,2019,0.715,715000
231,231,TO,Tonga,Country,2019,0.099,99000
232,232,TV,Tuvalu,Country,2019,0.010,10000


### Creating this list at the end since instructions were poorly written

In [16]:
regionList = [0] * 234

i = 0

for j in keepLater['index'][2:]:
    for i in WPDS_2020_data['index']:
        if i > j:
            regionList[i] = j
#regionList

[0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 58,
 58,
 58,
 58,
 58,
 58,
 64,
 64,
 64,
 67,
 68,
 68,
 68,
 68,
 68,
 68,
 68,
 68,
 68,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 109,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 129,
 129,
 129,
 129,
 129,
 129,
 135,
 135,
 135,
 135,
 135,
 135,
 135,
 135,
 135,
 135,
 145,
 145,
 145,
 145,
 145,
 145,
 145,
 145,
 145,
 145,
 145,
 145,
 157,
 157,
 157,
 157,
 157,
 157,
 157,
 157,
 157,
 166,
 167,
 167,
 167,
 167,
 167,
 167,
 167,
 167,
 167,
 167,
 167,
 167,
 179,
 179,
 179,
 179,
 179,
 179,
 179,

In [19]:
WPDS_2020_data['regionInd'] = regionList
WPDS_2020_data

,index,FIPS,Name,Type,TimeFrame,Data (M),Population,regionInd
0,0,WORLD,WORLD,World,2019,7772.850,7772850000,0
1,1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000,0
2,2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,0
3,3,DZ,Algeria,Country,2019,44.357,44357000,2
4,4,EG,Egypt,Country,2019,100.803,100803000,2
...,...,...,...,...,...,...,...,...
229,229,WS,Samoa,Country,2019,0.200,200000,216
230,230,SB,Solomon Islands,Country,2019,0.715,715000,216
231,231,TO,Tonga,Country,2019,0.099,99000,216
232,232,TV,Tuvalu,Country,2019,0.010,10000,216


### Setting up endpoint and function to read API

In [20]:
url_endpoint = "https://ores.wikimedia.org/v3/scores/enwiki/?models=articlequality&revids={rev_id}"

headers = {
    'User-Agent': 'https://github.com/Samperebikovsky',
    'From': 'sampereb@uw.edu'
}

def api_call(endpoint, rev_id):
    #print(rev_id)
    call = requests.get(endpoint.format(**rev_id), headers=headers)
    info = call.json()
    return info

### Creating list of lists for checking 50 rev_ids at a time instead of all at once

In [21]:
article_results = []
rev_ids = new_page['rev_id'].to_list()

fullList = "|".join([str(i) for i in rev_ids])
#print(fullList)

chunk_size = 500
splitList = [fullList[i:i+chunk_size] for i in range(0, len(fullList), chunk_size)]

finRevs = [i[:-1] for i in splitList]
#finRevs

### Function to accomplish the reading in batches

In [23]:
def rePreds(revList):
    parms = {"project": "enwiki", "model":"articlequality", "rev_id": revList}
    #print(parms)
    resp = api_call(url_endpoint,parms)
    preds = []
    rid = []
    
    for i in revList.split('|'):
        preds.append(resp.get('enwiki',{}).get('scores', {})
                          .get(i,{}).get("articlequality").get("score", {}).get("prediction", np.nan))
        rid.append(i)
    return preds,rid


### Running functions

In [24]:
fullPredictions = []
fullIds = []
for ids in finRevs:
    out,nid = rePreds(ids)
    fullPredictions.append(out)
    fullIds.append(nid)

In [25]:
#fullPredictions

In [26]:
#fullIds

### Flatten list of lists into one list to add to dataframe

In [27]:
flatPreds = [item for sublist in fullPredictions for item in sublist]
flatIds = [int(item) for sublist in fullIds for item in sublist]

In [28]:
predDf = pd.DataFrame()
predDf['rev_id'] = flatIds
predDf['ArtPred'] = flatPreds

In [29]:
#predDf
#new_page

## Step 3: Combining the Datasets

In [30]:
WPDS_2020_data = WPDS_2020_data.rename(columns={'Name': 'country'})
WPDS_2020_data

,index,FIPS,country,Type,TimeFrame,Data (M),Population,regionInd
0,0,WORLD,WORLD,World,2019,7772.850,7772850000,0
1,1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000,0
2,2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,0
3,3,DZ,Algeria,Country,2019,44.357,44357000,2
4,4,EG,Egypt,Country,2019,100.803,100803000,2
...,...,...,...,...,...,...,...,...
229,229,WS,Samoa,Country,2019,0.200,200000,216
230,230,SB,Solomon Islands,Country,2019,0.715,715000,216
231,231,TO,Tonga,Country,2019,0.099,99000,216
232,232,TV,Tuvalu,Country,2019,0.010,10000,216


In [31]:
fullData = pd.merge(new_page, predDf, on="rev_id", how = 'left')
fullData = pd.merge(fullData, WPDS_2020_data, on=['country'], how = 'left')
fullData

,page,country,rev_id,ArtPred,index,FIPS,Type,TimeFrame,Data (M),Population,regionInd
0,Bir I of Kanem,Chad,355319463,Stub,52.0,TD,Country,2019.0,16.877,1.687700e+07,48.0
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,Stub,122.0,PS,Country,2019.0,5.008,5.008000e+06,110.0
2,Yos Por,Cambodia,393822005,Stub,147.0,KH,Country,2019.0,15.497,1.549700e+07,145.0
3,Julius Gregr,Czech Republic,395521877,Stub,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Edvard Gregr,Czech Republic,395526568,Stub,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
46696,Yahya Jammeh,Gambia,807482007,GA,15.0,GM,Country,2019.0,2.417,2.417000e+06,10.0
46697,Lucius Fairchild,United States,807483006,C,66.0,US,Country,2019.0,329.878,3.298780e+08,64.0
46698,Fahd of Saudi Arabia,Saudi Arabia,807483153,GA,124.0,SA,Country,2019.0,35.041,3.504100e+07,110.0
46699,Francis Fessenden,United States,807483270,C,66.0,US,Country,2019.0,329.878,3.298780e+08,64.0


### Limiting columns

In [32]:
fullData = fullData[['country', 'page', 'rev_id', 'ArtPred', 'Population','regionInd']]
fullData

,country,page,rev_id,ArtPred,Population,regionInd
0,Chad,Bir I of Kanem,355319463,Stub,1.687700e+07,48.0
1,Palestinian Territory,Information Minister of the Palestinian Nation...,393276188,Stub,5.008000e+06,110.0
2,Cambodia,Yos Por,393822005,Stub,1.549700e+07,145.0
3,Czech Republic,Julius Gregr,395521877,Stub,NaN,NaN
4,Czech Republic,Edvard Gregr,395526568,Stub,NaN,NaN
...,...,...,...,...,...,...
46696,Gambia,Yahya Jammeh,807482007,GA,2.417000e+06,10.0
46697,United States,Lucius Fairchild,807483006,C,3.298780e+08,64.0
46698,Saudi Arabia,Fahd of Saudi Arabia,807483153,GA,3.504100e+07,110.0
46699,United States,Francis Fessenden,807483270,C,3.298780e+08,64.0


### Removing null here

In [34]:
nullData = fullData[fullData.isna().any(axis=1)]
nullData = nullData.rename(columns={'page': 'article_name', 'rev_id':'revision_id', 'ArtPred':'article_quality_est.'
                                    , 'Population': 'population'})
nullData

,country,article_name,revision_id,article_quality_est.,population,regionInd
3,Czech Republic,Julius Gregr,395521877,Stub,NaN,NaN
4,Czech Republic,Edvard Gregr,395526568,Stub,NaN,NaN
14,Poland,List of politicians in Poland,516633096,NaN,3.836800e+07,189.0
21,Vanuatu,Tingtingru,550682925,NaN,3.210000e+05,216.0
31,Salvadoran,Timoteo Menéndez,566504165,Stub,NaN,NaN
...,...,...,...,...,...,...
46596,Omani,Haitham bin Tariq Al Said,807451487,Stub,NaN,NaN
46600,Chechen,Ruslan Yamadayev,807454176,B,NaN,NaN
46639,Czech Republic,Vladimír Špidla,807462107,C,NaN,NaN
46686,Philippines,King Gutierrez,807479587,NaN,1.095810e+08,145.0


In [35]:
predData = fullData.dropna()
predData = predData.rename(columns={'page': 'article_name', 'rev_id':'revision_id', 'ArtPred':'article_quality_est.'
                                    , 'Population': 'population'})
predData

,country,article_name,revision_id,article_quality_est.,population,regionInd
0,Chad,Bir I of Kanem,355319463,Stub,16877000.0,48.0
1,Palestinian Territory,Information Minister of the Palestinian Nation...,393276188,Stub,5008000.0,110.0
2,Cambodia,Yos Por,393822005,Stub,15497000.0,145.0
5,Canada,Robert Douglas Cook,401577829,Stub,38190000.0,64.0
6,Egypt,List of Grand Viziers of Egypt,442937236,Stub,100803000.0,2.0
...,...,...,...,...,...,...
46695,United States,Hal Bidlack,807481636,C,329878000.0,64.0
46696,Gambia,Yahya Jammeh,807482007,GA,2417000.0,10.0
46697,United States,Lucius Fairchild,807483006,C,329878000.0,64.0
46698,Saudi Arabia,Fahd of Saudi Arabia,807483153,GA,35041000.0,110.0


### Output csv files here

In [37]:
predData[['country', 'article_name', 'revision_id', 'article_quality_est.', 'population']].to_csv('wp_wpds_politicians_by_country.csv')
nullData[['country', 'article_name', 'revision_id', 'article_quality_est.', 'population']].to_csv('wp_wpds_countries-no_match.csv')

## Step 4: Analysis

In [84]:
predData[['country', 'article_name', 'article_quality_est.', 'population']].to_csv('final_article_data.csv')
predData

,country,article_name,revision_id,article_quality_est.,population,regionInd
0,Chad,Bir I of Kanem,355319463,Stub,16877000.00000000,48.00000000
1,Palestinian Territory,Information Minister of the Palestinian Nation...,393276188,Stub,5008000.00000000,110.00000000
2,Cambodia,Yos Por,393822005,Stub,15497000.00000000,145.00000000
5,Canada,Robert Douglas Cook,401577829,Stub,38190000.00000000,64.00000000
6,Egypt,List of Grand Viziers of Egypt,442937236,Stub,100803000.00000000,2.00000000
...,...,...,...,...,...,...
46695,United States,Hal Bidlack,807481636,C,329878000.00000000,64.00000000
46696,Gambia,Yahya Jammeh,807482007,GA,2417000.00000000,10.00000000
46697,United States,Lucius Fairchild,807483006,C,329878000.00000000,64.00000000
46698,Saudi Arabia,Fahd of Saudi Arabia,807483153,GA,35041000.00000000,110.00000000


### Queries to find number of articles by country

In [42]:
queryProp = """SELECT country, CAST(CAST(population AS FLOAT) AS bigint) AS population, COUNT(*) , (COUNT(*)/population *100) as [ArticleProportion (%)], regionInd as [index] FROM predData 
                    GROUP BY country, population, regionInd;"""

queryQualProp = """SELECT country, CAST(CAST(population AS FLOAT) AS bigint) AS population, COUNT(*) , CAST((COUNT(*)/population *100) AS DECIMAL (8,8)) as [QualityArticleProportion (%)], regionInd as [index] FROM predData WHERE
                    [article_quality_est.] = 'GA' OR [article_quality_est.] = 'FA'
                    GROUP BY country, population, regionInd;"""

# CAST(CAST('7.54001e+006'  AS FLOAT) AS bigint)

In [43]:
prop = sqldf.run(queryProp)

qProp = sqldf.run(queryQualProp)

In [44]:
prop = prop.sort_values('ArticleProportion (%)', ascending = False)
prop

#all articles

,country,population,COUNT(*),ArticleProportion (%),index
169,Tuvalu,10000,54,0.540000,216.0
117,Nauru,11000,52,0.472727,216.0
138,San Marino,34000,81,0.238235,200.0
110,Monaco,38000,40,0.105263,179.0
95,Liechtenstein,39000,28,0.071795,179.0
...,...,...,...,...,...
51,Ethiopia,114916000,101,0.000088,27.0
176,Uzbekistan,34174000,28,0.000082,129.0
34,China,1402385000,1129,0.000081,157.0
72,Indonesia,271739000,209,0.000077,145.0


In [46]:
qProp = (qProp.sort_values('QualityArticleProportion (%)', ascending = False))
qProp.reset_index(inplace = True, drop = True)
#pd.options.display.float_format = '{:.8f}'.format
qProp

# high quality articles

,country,population,COUNT(*),QualityArticleProportion (%),index
0,Tuvalu,10000,4,0.04000000,216.00000000
1,Dominica,72000,1,0.00138889,77.00000000
2,Vanuatu,321000,3,0.00093458,216.00000000
3,Iceland,368000,2,0.00054348,167.00000000
4,Ireland,5003000,25,0.00049970,167.00000000
...,...,...,...,...,...
141,Bangladesh,169809000,3,0.00000177,135.00000000
142,Ethiopia,114916000,2,0.00000174,27.00000000
143,Tanzania,59734000,1,0.00000167,27.00000000
144,Nigeria,206140000,2,0.00000097,10.00000000


In [ ]:
########################################################
#  Region Calculations

### Modifying data for Regional results

In [63]:
regionData = pd.merge(prop, keepLater, on="index", how = 'left')
regionData = regionData[['country', 'Population', 'COUNT(*)', 'Name' ]]
regionData.rename(columns = {'COUNT(*)':'numArticles'}, inplace = True)
regionData

,country,Population,numArticles,Name
0,Tuvalu,43155000,54,OCEANIA
1,Nauru,43155000,52,OCEANIA
2,San Marino,153251000,81,SOUTHERN EUROPE
3,Monaco,195479000,40,WESTERN EUROPE
4,Liechtenstein,195479000,28,WESTERN EUROPE
...,...,...,...,...
178,Ethiopia,444970000,101,EASTERN AFRICA
179,Uzbekistan,74961000,28,CENTRAL ASIA
180,China,1641063000,1129,EAST ASIA
181,Indonesia,661845000,209,SOUTHEAST ASIA


In [62]:
regionDataQ = pd.merge(qProp, keepLater, on="index", how = 'left')
regionDataQ = regionDataQ[['country', 'Population', 'COUNT(*)', 'Name' ]]
regionDataQ.rename(columns = {'COUNT(*)':'numArticlesQ'}, inplace = True)
regionDataQ

,country,Population,numArticlesQ,Name
0,Tuvalu,43155000,4,OCEANIA
1,Dominica,43233000,1,CARIBBEAN
2,Vanuatu,43155000,3,OCEANIA
3,Iceland,105990000,2,NORTHERN EUROPE
4,Ireland,105990000,25,NORTHERN EUROPE
...,...,...,...,...
141,Bangladesh,1967131000,3,SOUTH ASIA
142,Ethiopia,444970000,2,EASTERN AFRICA
143,Tanzania,444970000,1,EASTERN AFRICA
144,Nigeria,401115000,2,WESTERN AFRICA


### Queries for our regional data and counting number of articles

In [69]:
regQ1 = """SELECT Name, Population, SUM(numArticles) as numArticle, (SUM(numArticles)*100)/CAST(Population AS FLOAT) as articleProportion
FROM regionData GROUP BY Name, Population; """

regQ2 = """SELECT Name, Population, SUM(numArticlesQ) as numArticleQ, (SUM(numArticlesQ)*100)/CAST(Population AS FLOAT) as articleProportion
FROM regionDataQ GROUP BY Name, Population; """


regionProp = sqldf.run(regQ1)
regionPropQ = sqldf.run(regQ2)

In [72]:
pd.options.display.float_format = '{:.8f}'.format
regionProp = regionProp.sort_values('articleProportion', ascending = False)
regionProp

,Name,Population,numArticle,articleProportion
10,OCEANIA,43155000,3126,0.00724366
9,NORTHERN EUROPE,105990000,3763,0.00355033
15,SOUTHERN EUROPE,153251000,3710,0.00242087
18,WESTERN EUROPE,195479000,4560,0.00233273
0,CARIBBEAN,43233000,695,0.00160757
5,EASTERN EUROPE,291902000,3732,0.00127851
14,SOUTHERN AFRICA,67732000,634,0.00093604
17,WESTERN ASIA,280927000,2563,0.00091234
1,CENTRAL AMERICA,178611000,1543,0.00086389
11,SOUTH AMERICA,429191000,3032,0.00070645


In [73]:
regionPropQ = regionPropQ.sort_values('articleProportion', ascending = False)
regionPropQ

,Name,Population,numArticleQ,articleProportion
10,OCEANIA,43155000,63,0.00014599
9,NORTHERN EUROPE,105990000,102,0.00009624
15,SOUTHERN EUROPE,153251000,74,0.00004829
5,EASTERN EUROPE,291902000,118,0.00004042
17,WESTERN ASIA,280927000,89,0.00003168
0,CARIBBEAN,43233000,13,0.00003007
18,WESTERN EUROPE,195479000,56,0.00002865
8,NORTHERN AMERICA,368193000,104,0.00002825
14,SOUTHERN AFRICA,67732000,9,0.00001329
1,CENTRAL AMERICA,178611000,23,0.00001288


## Step 5: Results

#### #1 Top 10 Countries by proportion of articles/population

In [78]:
#1
prop.head(10)

,country,population,COUNT(*),ArticleProportion (%),index
169,Tuvalu,10000,54,0.54000000,216.00000000
117,Nauru,11000,52,0.47272727,216.00000000
138,San Marino,34000,81,0.23823529,200.00000000
110,Monaco,38000,40,0.10526316,179.00000000
95,Liechtenstein,39000,28,0.07179487,179.00000000
104,Marshall Islands,57000,37,0.06491228,216.00000000
164,Tonga,99000,63,0.06363636,216.00000000
70,Iceland,368000,201,0.05461957,167.00000000
3,Andorra,82000,34,0.04146341,200.00000000
52,Federated States of Micronesia,106000,36,0.03396226,216.00000000


#### #2 Bottom 10 countries by proportion of articles/population

In [79]:
#2
prop.tail(10)

,country,population,COUNT(*),ArticleProportion (%),index
13,Bangladesh,169809000,317,0.00018668,135.00000000
114,Mozambique,31166000,58,0.00018610,27.00000000
162,Thailand,66534000,112,0.00016833,145.00000000
84,"Korea, North",25779000,36,0.00013965,157.00000000
181,Zambia,18384000,25,0.00013599,27.00000000
51,Ethiopia,114916000,101,0.00008789,27.00000000
176,Uzbekistan,34174000,28,0.00008193,129.00000000
34,China,1402385000,1129,0.00008051,157.00000000
72,Indonesia,271739000,209,0.00007691,145.00000000
71,India,1400100000,968,0.00006914,135.00000000


#### #3 Top 10 Countries by proportion of quality articles/population

In [80]:
#3
qProp.head(10)

,country,population,COUNT(*),QualityArticleProportion (%),index
0,Tuvalu,10000,4,0.04000000,216.00000000
1,Dominica,72000,1,0.00138889,77.00000000
2,Vanuatu,321000,3,0.00093458,216.00000000
3,Iceland,368000,2,0.00054348,167.00000000
4,Ireland,5003000,25,0.00049970,167.00000000
5,Montenegro,622000,2,0.00032154,200.00000000
6,Martinique,356000,1,0.00028090,77.00000000
7,Bhutan,730000,2,0.00027397,135.00000000
8,New Zealand,4987000,13,0.00026068,216.00000000
9,Romania,19241000,42,0.00021828,189.00000000


#### #4 Bottom 10 countries by proportion of quality articles/population

In [81]:
#4
qProp.tail(10)

,country,population,COUNT(*),QualityArticleProportion (%),index
136,China,1402385000,40,0.00000285,157.00000000
137,Brazil,211812000,6,0.00000283,95.00000000
138,Morocco,35952000,1,0.00000278,2.00000000
139,Uganda,45741000,1,0.00000219,27.00000000
140,Colombia,49444000,1,0.00000202,95.00000000
141,Bangladesh,169809000,3,0.00000177,135.00000000
142,Ethiopia,114916000,2,0.00000174,27.00000000
143,Tanzania,59734000,1,0.00000167,27.00000000
144,Nigeria,206140000,2,0.00000097,10.00000000
145,India,1400100000,13,0.00000093,135.00000000


#### #5 Proportion of articles/population by region 

In [82]:
#5
regionProp

,Name,Population,numArticle,articleProportion
10,OCEANIA,43155000,3126,0.00724366
9,NORTHERN EUROPE,105990000,3763,0.00355033
15,SOUTHERN EUROPE,153251000,3710,0.00242087
18,WESTERN EUROPE,195479000,4560,0.00233273
0,CARIBBEAN,43233000,695,0.00160757
5,EASTERN EUROPE,291902000,3732,0.00127851
14,SOUTHERN AFRICA,67732000,634,0.00093604
17,WESTERN ASIA,280927000,2563,0.00091234
1,CENTRAL AMERICA,178611000,1543,0.00086389
11,SOUTH AMERICA,429191000,3032,0.00070645


#### #6 Proportion of quality articles/population by region 

In [83]:
#6
regionPropQ

,Name,Population,numArticleQ,articleProportion
10,OCEANIA,43155000,63,0.00014599
9,NORTHERN EUROPE,105990000,102,0.00009624
15,SOUTHERN EUROPE,153251000,74,0.00004829
5,EASTERN EUROPE,291902000,118,0.00004042
17,WESTERN ASIA,280927000,89,0.00003168
0,CARIBBEAN,43233000,13,0.00003007
18,WESTERN EUROPE,195479000,56,0.00002865
8,NORTHERN AMERICA,368193000,104,0.00002825
14,SOUTHERN AFRICA,67732000,9,0.00001329
1,CENTRAL AMERICA,178611000,23,0.00001288


## Writeup and Reflection

I think it was partially surprising to see larger countries never appearead in the top 10. But after reflection this makes sense since people who deserve wikipedia articles scales slowly while population scales somewhat exponentially. Seeing (relatively) well known countries in the bottom 10 while the top 10 were often populated by tiny island nations and less well known places made me realize that you only need one wikipedia editor to take an interest in a country like Vanuatu to publish a couple articles and they will be propelled to the top of the (all article) rankings. However, it is still not clear to me why these smaller places have such a (relatively) high amount of quality articles. This means there is enough interest to write a lot about some particular people in a different, and very small, country. Like the fact that Emmanuel Macron's page is only B-rated I thought meant that a majority of smaller countries would likely have no articles of higher quality than B, but I was mistaken. Or maybe the predictor was mistaken? I think it would be simple to extract the true value/quality of the wikipedia article so I felt this was unnecessary. 

I expected the top 10 countries in both the 'all articles' and 'quality articles' tables to be mostly different which seems to be the case. I think it was interesting to think critically about the assignment and question why we did some of the the things we did. I think most of the biases come from using english wikipedia and the article quality predictor. I elaborate further below. 


(This reflection covers topics from question # 1,2,3,7)

There were some interesting things going on with this data from the beginning. I think one of the potential sources of bias is the fact that we were using english wikipedia. Presumably this would mean that there are more articles about people who speak english and than people who are mostly isolated from the english speaking world. In addition, the only people writing the articles and making the quality better are also english speaking which almost certainly adds an element of bias to this assignment. 

Something else to think about is how we used a machine learning model to predict the quality of the article. Now this could be skewing our results by a significant margin. Ideally for an analysis of article number/quality we would like to use the true values, which should be obtainible by using Wikipedia's API. This would remove the uncertainty inherently associated with a machine learning model. 

Something interesting I noticed is that it seems to be the case that the population of a country grows much faster than the amount of people with wikipedia articles grows. This can be seen when countries like India, China, and Brazil often being in the bottom 10 of articles/population proportions despite having some very high populations. In the last two tables we can see regions in Asia and Africa consistently rank towards the bottom while regions like Oceania, Europe, and Americas are often towrds the top in terms of proportion of wikipedia articles/population. This lends credibility to the idea that using english wikipedia skews our results.

As previously stated I think one of the best ways to combat the problems in this analysis is to take the true values of article ranking instead of using machine learning to predict them. I also think if that is not feasible then taking data from all sources of wikipedia (not just english) and combining the data based on the number of people that have articles could also make it less biased towards the western world.